In [13]:
import pandas as pd
import numpy as np
from data import Urg2022_pub, UrgP2022, Urg2022_priv_luc, Urg2022_priv_non_luc

In [ ]:
###---Productivité des médecins---

#Nombre de passages aux urgences par médecin en 2022 selon le secteur 

#Extraction du nombre d'ETP de médecins
Medecins2022 = UrgP2022[UrgP2022["PERSO"].isin(["M1000","M2000","M3020"])]
Medecins2022["ETPSAL"].fillna(0)
Medecins2022["EFFLIB"].fillna(0)
ETP_Medecins2022 = Medecins2022.groupby("FI",as_index=False)[["ETPSAL","EFFLIB"]].sum()
ETP_Medecins2022["tot"] = ETP_Medecins2022["EFFLIB"] + ETP_Medecins2022["ETPSAL"]
ETP_Medecins2022.describe()


In [ ]:

#Rassemble les médecins ETP avec le nombre de passages aux urgences selon le type d'établissement
#et calcule le ratio passages aux urgences/ETP médecins
Passages_pub = Urg2022_pub[["FI","PASSU"]]
Passages_pub = Passages_pub.merge(ETP_Medecins2022, left_on="FI", right_on="FI", how="left")
Passages_pub.dropna(inplace = True)
Passages_pub["ratio"] = Passages_pub["PASSU"]/Passages_pub["tot"]
Passages_pub["ratio"].describe()


In [ ]:
#privé lucratif
Passages_priv_luc = Urg2022_priv_luc[["FI","PASSU"]]
Passages_priv_luc = Passages_priv_luc.merge(ETP_Medecins2022, left_on="FI", right_on="FI", how="left")
Passages_priv_luc.dropna(inplace=TRUE)
Passages_priv_luc["ratio"] = Passages_priv_luc["PASSU"]/Passages_priv_luc["tot"]
Passages_priv_luc["ratio"].describe()



In [ ]:
#privé non lucratif
Passages_priv_nonluc = Urg2022_priv_non_luc[["FI","PASSU"]]
Passages_priv_nonluc = Passages_priv_nonluc.merge(ETP_Medecins2022, left_on="FI", right_on="FI", how="left")
Passages_priv_nonluc.dropna(inplace=True)
Passages_priv_nonluc["ratio"] = Passages_priv_nonluc["PASSU"]/Passages_priv_nonluc["tot"]
Passages_priv_nonluc["ratio"].describe()

In [ ]:
###--- Productivité des IDE ---

#Secteur public
Passages_pub = Urg2022_pub[["FI","PASSU","HIDE"]]
Passages_pub.groupby("FI",as_index=False)[["PASSU","HIDE"]].sum()
Passages_pub = Passages_pub.replace(0, np.nan).dropna()
Passages_pub["ratio"] = Passages_pub["PASSU"]/Passages_pub["HIDE"]
Passages_pub["ratio"].describe()

count     523.000000
mean       60.382621
std       102.119381
min         3.671102
25%        36.703995
50%        45.609489
75%        59.981821
max      1696.361111
Name: ratio, dtype: float64

In [16]:

#Secteur privé lucratif
Passages_priv_luc = Urg2022_priv_luc[["FI","PASSU","HIDE"]]
Passages_priv_luc.groupby("FI",as_index=False)[["PASSU","HIDE"]].sum()
Passages_priv_luc = Passages_priv_luc.replace(0, np.nan).dropna()
Passages_priv_luc["ratio"] = Passages_priv_luc["PASSU"]/Passages_priv_luc["HIDE"]
Passages_priv_luc["ratio"].describe()


count     121.000000
mean       71.466510
std       138.748197
min         6.250000
25%        43.501656
50%        53.571429
75%        68.755952
max      1503.411765
Name: ratio, dtype: float64

In [17]:

#Secteur privé non lucratif
Passages_priv_non_luc = Urg2022_priv_non_luc[["FI","PASSU","HIDE"]]
Passages_priv_non_luc.groupby("FI",as_index=False)[["PASSU","HIDE"]].sum()
Passages_priv_luc = Passages_priv_non_luc.replace(0, np.nan).dropna()
Passages_priv_non_luc["ratio"] = Passages_priv_non_luc["PASSU"]/Passages_priv_non_luc["HIDE"]
Passages_priv_non_luc["ratio"].describe()

C:\Users\roman\AppData\Local\Temp\ipykernel_6004\2980174334.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Passages_priv_non_luc["ratio"] = Passages_priv_non_luc["PASSU"]/Passages_priv_non_luc["HIDE"]


count     36.000000
mean      68.478455
std       86.184945
min        1.695901
25%       36.107463
50%       47.818013
75%       69.577142
max      522.714286
Name: ratio, dtype: float64